<a href="https://colab.research.google.com/github/k589k589/DeepLearning/blob/main/Neural_style_transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

loss = (distance(style(reference_image) - style(combination_image)) +
        distance(content(original_image) - content(combination_image)))

In [ ]:

from tensorflow import keras

base_image_path = keras.utils.get_file(
    "sf.jpg", origin="https://img-datasets.s3.amazonaws.com/sf.jpg")
style_reference_image_path = keras.utils.get_file(
    "starry_night.jpg", origin="https://img-datasets.s3.amazonaws.com/starry_night.jpg")

original_width, original_height = keras.utils.load_img(base_image_path).size
img_height = 400
img_width = round(original_width * img_height / original_height)

958464/943128 [==============================] - 0s 0us/step


# Auxiliary functions
You need some auxiliary functions for loading, preprocessing, and postprocessing the images that go in and out of the VGG19 convnet.

In [ ]:
import numpy as np

def preprocess_image(image_path):
    img = keras.utils.load_img(
        image_path, target_size=(img_height, img_width))
    img = keras.utils.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = keras.applications.vgg19.preprocess_input(img)
    return img

def deprocess_image(img):
    img = img.reshape((img_height, img_width, 3))
    img[:, :, 0] += 103.939
    img[:, :, 1] += 116.779
    img[:, :, 2] += 123.68
    img = img[:, :, ::-1]
    img = np.clip(img, 0, 255).astype("uint8")
    return img

# 加載預訓練的 VGG19 網絡並使用它來定義特徵提取器

In [ ]:

model = keras.applications.vgg19.VGG19(weights="imagenet", include_top=False)

outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])
feature_extractor = keras.Model(inputs=model.inputs, outputs=outputs_dict)

80150528/80134624 [==============================] - 1s 0us/step


# define content loss

In [ ]:

def content_loss(base_img, combination_img):
    return tf.reduce_sum(tf.square(combination_img - base_img))

# define style loss
它使用一個輔助函數來計算輸入矩陣的 Gram 矩陣：在原始特徵矩陣中找到的相關性的映射。

In [ ]:
def gram_matrix(x):
    x = tf.transpose(x, (2, 0, 1))
    features = tf.reshape(x, (tf.shape(x)[0], -1))
    gram = tf.matmul(features, tf.transpose(features))
    return gram

def style_loss(style_img, combination_img):
    S = gram_matrix(style_img)
    C = gram_matrix(combination_img)
    channels = 3
    size = img_height * img_width
    return tf.reduce_sum(tf.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

# the total variation loss
它對生成的組合圖像的像素進行操作。它鼓勵生成圖像的空間連續性，從而避免過度像素化的結果。您可以將其解釋為正則化損失。

In [ ]:
def total_variation_loss(x):
    a = tf.square(
        x[:, : img_height - 1, : img_width - 1, :] - x[:, 1:, : img_width - 1, :]
    )
    b = tf.square(
        x[:, : img_height - 1, : img_width - 1, :] - x[:, : img_height - 1, 1:, :]
    )
    return tf.reduce_sum(tf.pow(a + b, 1.25))


您最小化的損失是這三個損失的加權平均值。為了計算內容損失，你只使用一個上層——block5_conv2層——而對於樣式損失，你使用一個跨越低層和高層的層列表。最後添加總變異損失。

根據您使用的樣式參考圖像和內容圖像，您可能需要調整content_weight係數（內容損失對總損失的貢獻）。越高content_weight意味著目標內容在生成的圖像中將更容易識別。

# Defining the final loss that you'll minimize

In [ ]:
style_layer_names = [
    "block1_conv1",
    "block2_conv1",
    "block3_conv1",
    "block4_conv1",
    "block5_conv1",
]
content_layer_name = "block5_conv2"
total_variation_weight = 1e-6
style_weight = 1e-6
content_weight = 2.5e-8

def compute_loss(combination_image, base_image, style_reference_image):
    input_tensor = tf.concat(
        [base_image, style_reference_image, combination_image], axis=0
    )
    features = feature_extractor(input_tensor)
    loss = tf.zeros(shape=())
    layer_features = features[content_layer_name]
    base_image_features = layer_features[0, :, :, :]
    combination_features = layer_features[2, :, :, :]
    loss = loss + content_weight * content_loss(
        base_image_features, combination_features
    )
    for layer_name in style_layer_names:
        layer_features = features[layer_name]
        style_reference_features = layer_features[1, :, :, :]
        combination_features = layer_features[2, :, :, :]
        style_loss_value = style_loss(
          style_reference_features, combination_features)
        loss += (style_weight / len(style_layer_names)) * style_loss_value

    loss += total_variation_weight * total_variation_loss(combination_image)
    return loss

# 設置梯度下降過程
最後，讓我們設置梯度下降過程。在原來的 Gatys 等人。論文中，優化是使用 L-BFGS 算法執行的，但這在 TensorFlow 中不可用，因此我們將只使用SGD優化器進行小批量梯度下降。我們將利用您以前從未見過的優化器功能：學習率計劃。我們將使用它逐漸將學習率從非常高的值（100）降低到更小的最終值（大約 20）。這樣，我們將在訓練的早期階段取得快速進展，然後在接近損失最小值時更加謹慎地進行。

In [ ]:
import tensorflow as tf

@tf.function
def compute_loss_and_grads(combination_image, base_image, style_reference_image):
    with tf.GradientTape() as tape:
        loss = compute_loss(combination_image, base_image, style_reference_image)
    grads = tape.gradient(loss, combination_image)
    return loss, grads

optimizer = keras.optimizers.SGD(
    keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=100.0, decay_steps=100, decay_rate=0.96
    )
)

base_image = preprocess_image(base_image_path)
style_reference_image = preprocess_image(style_reference_image_path)
combination_image = tf.Variable(preprocess_image(base_image_path))

iterations = 4000
for i in range(1, iterations + 1):
    loss, grads = compute_loss_and_grads(
        combination_image, base_image, style_reference_image
    )
    optimizer.apply_gradients([(grads, combination_image)])
    if i % 100 == 0:
        print(f"Iteration {i}: loss={loss:.2f}")
        img = deprocess_image(combination_image.numpy())
        fname = f"combination_image_at_iteration_{i}.png"
        keras.utils.save_img(fname, img)

KeyboardInterrupt: ignored

Style transfer consists of creating a new image that preserves the contents of a target image while also capturing the style of a reference image.

Content can be captured by the high-level activations of a convnet.

Style can be captured by the internal correlations of the activations of different layers of a convnet.

Hence, deep learning allows style transfer to be formulated as an optimization process using a loss defined with a pretrained convnet.

Starting from this basic idea, many variants and refinements are possible.

風格遷移包括創建一個新圖像，該圖像保留目標圖像的內容，同時還捕獲參考圖像的風格。

內容可以通過 convnet 的高級激活來捕獲。

風格可以通過卷積網絡不同層的激活的內部相關性來捕獲。

因此，深度學習允許將風格遷移表述為使用預訓練卷積網絡定義的損失的優化過程。

從這個基本思想開始，許多變體和改進都是可能的。